# MNIST Embeddings

## Imports and Hyper-parameters 

In [1]:
import torch 
import os 
import numpy as np 
from torchvision import transforms 
from torchvision.datasets import MNIST 
from torchvision import models 

print(os.getcwd())
root = os.getcwd()

from DeepFeatures import DeepFeatures
from mnist_net import Net

/home/praveens/Desktop/synthetic_biometrics/visualize_embeddings


In [2]:
batch_size = 128
data_folder = root + '/MNIST'
device = 'cpu'
model_path = './mnist_net.pth'

## Create Dataloader

In [3]:
transformations = transforms.Compose([transforms.Resize((221, 221)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485], std=[0.229])
                                    ])

mnist_data = MNIST(root=r'./MNIST',
                   download=False, train=False,
                   transform=transformations)

data_loader = torch.utils.data.DataLoader(mnist_data,
                                          batch_size=batch_size,
                                          shuffle=True)

In [4]:
# list(mnist_net.named_parameters())

## Initialize Tensorboard Logging Class 

In [5]:
mnist_net = Net().to(device)
mnist_net.load_state_dict(torch.load(model_path))  # load saved model 

mnist_net.eval()

Net(
  (conv1): Conv2d(1, 8, kernel_size=(2, 2), stride=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(2, 2), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=3, bias=True)
  (fc4): Linear(in_features=3, out_features=3, bias=True)
  (fc5): Linear(in_features=3, out_features=10, bias=True)
)

In [6]:
deep_features = DeepFeatures(model=mnist_net)

In [7]:
def find_class_embeddings(deep_features, data_loader, class1, class2):
    images, labels = next(iter(data_loader))
    e = deep_features.generate_embeddings(images).clone().detach()
    
    x1 = torch.zeros(3)
    class1 = torch.tensor(class1)
    class1_count = 0

    x2 = torch.zeros(3)
    class2 = torch.tensor(class2)
    class2_count = 0

    for index, (image, label) in enumerate(zip(images, labels)):
        if class2_count == 1 and class2_count == 1:
            break 
        
        if (not class1_count) and label == class1:
            x1 += e[index]/torch.norm(e[index])
            class1_count += 1
        if (not class2_count) and label == class2:
            x2 += e[index]/torch.norm(e[index])
            class2_count += 1
    
    print('sum of class ' + str(class1.numpy()) + ' : ', x1)
    print('sum of class '  + str(class2.numpy()) + ' : ', x2)
    
    x1 = x1/class1_count
    x2 = x2/class2_count
    
    return x1, x2

In [8]:
x1, x2 = find_class_embeddings(deep_features, data_loader, 1, 0)
x1, x2

sum of class 1 :  tensor([ 0.1610,  0.0786, -0.9838])
sum of class 0 :  tensor([0.7407, 0.5354, 0.4059])


(tensor([ 0.1610,  0.0786, -0.9838]), tensor([0.7407, 0.5354, 0.4059]))

In [9]:
def check_if_orthogonal(nbv, q_i):
    is_orthogonal = True
    for q_n in nbv:
        if torch.abs(torch.dot(q_n, q_i)) >= torch.tensor(1e-05):   
            is_orthogonal = False   # has to be orthogonal to all of the basis vectors in the list 
    return is_orthogonal

In [10]:
def perform_weight_surgery(mnist_net, x1, x2, save_model=True):
    
    # calculate the difference between the 2 class vectors 
    d = (x2/torch.norm(x2)) - (x1/torch.norm(x1)) 
    d = d/torch.norm(d)
    weight_matrix = mnist_net.fc4.weight.clone().detach()  # weight matrix of the penultimate layer 
    
    def gram_schmidt(d, weight_matrix):
        new_basis_vectors = [d]

        m, n = weight_matrix.shape

        for i in range(n):

            a_i = weight_matrix[:, i].clone()
            q_i = 0

            for j, q_j in enumerate(new_basis_vectors):
                q_i -= (torch.dot(q_j, a_i) * q_j)
            
            q_i += a_i
            q_i = q_i / torch.norm(q_i)
            
            if check_if_orthogonal(new_basis_vectors, q_i):
                new_basis_vectors.append(q_i)
            
        return new_basis_vectors
    
    
    new_basis_vectors = gram_schmidt(d, weight_matrix)
    print('weight matrix :\n', weight_matrix)
    
    U = [nbv.reshape(-1, 1) for nbv in new_basis_vectors]
    U = torch.cat(U, axis=1)
#     Q, _ = torch.qr(U)
    print('unitary matrix :\n', U)
#     print('QR decomp :\n', Q)
    
    S = torch.eye(U.shape[0])
    S[0, 0] = 0
    print('projection matrix :\n', S)
    
    print('modified weight matrix :\n', weight_matrix @ U @ S @ U.T)
    
    mnist_net.fc4.weight = torch.nn.Parameter(weight_matrix @ U @ S @ U.T)
    
    if save_model: 
        path = './modified_mnist_net.pth'
        torch.save(mnist_net.state_dict(), path)

In [11]:
perform_weight_surgery(mnist_net, x1, x2, save_model=True)

weight matrix :
 tensor([[ 0.3257, -1.0893,  0.8606],
        [ 0.0483,  0.1397,  0.5508],
        [-1.0913,  0.2588,  0.4324]])
unitary matrix :
 tensor([[ 0.3684,  0.8079, -0.4600],
        [ 0.2903,  0.3701,  0.8825],
        [ 0.8832, -0.4586, -0.0982]])
projection matrix :
 tensor([[0., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
modified weight matrix :
 tensor([[ 0.1180, -1.2530,  0.3627],
        [-0.1524, -0.0184,  0.0696],
        [-1.1116,  0.2428,  0.3838]])


## Write Embeddings to Tensorboard

In [12]:
batch_images, batch_labels = next(iter(data_loader))
deep_features.write_embeddings(x=batch_images.to(device))

True

In [13]:
deep_features.create_tensorboard_log()

torch.Size([128, 3])
torch.Size([128, 1, 28, 28])


/home/praveens/Desktop/synthetic_biometrics/visualize_embeddings/DeepFeatures.py:86: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /home/builder/mc3/envs/pytorch-build/envs/pytorch-build/conda-bld/pytorch_1673601922403/work/torch/csrc/utils/tensor_new.cpp:201.)
  all_embeds = torch.Tensor(all_embeds)


In [14]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for images, labels in data_loader:
        images = images.to(device); labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = mnist_net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


In [15]:
print(f'Accuracy of the network on the '+ str(total) +' test images: '+ str(100 * correct // total)+ '%')

Accuracy of the network on the 10000 test images: 50%


In [16]:
# 1, 0 split (accuracy 45%)
# 1, 2 split (accuracy 47%)
# 1, 3 split (accuracy 46%)
# 1, 4 split (accuracy 36%)
# 1, 5 split (accuracy 39%)
# 1, 6 split (accuracy 52%) had to run twice 
# 1, 7 split (accuracy Not Possible %)
# 1, 8 split (accuracy  Not possible %)
# 1, 9 split (accuracy  37%)